# Identifier les textes évoquant le concept de république

TODO:
Premier pb ici : vérifier d'ou vient mais
- changement taille fichier entrée (check)
- ex avant de changer cleaning ave le join : 129769 / puis
- VÉRIF ÉCRITURE DU CSV QUI DOIT MERDER ????
- -> C'est le join avant qui visiblement introduit une merde
- mais ne change pas la structure du df avant l'export/reimport
- résolu en forçant le quote à l'export du csv

In [22]:
import pandas as pd

df = pd.read_csv(
    "../data/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)

In [23]:
# ICI ÇA MERDAIT ENTRE LA SORTIE JUSTE AVANT ET LA LECTURE ET IMPORT DU DF
# Resolu avec quotes
# Voir pourquoi et ou est le pb quand même dans les cols du join
df.shape

(129769, 59)

Logique de la tentative :
- regex
- mais exclure certains termes
- mais comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (les idées républicaines sont menacées par Les Républicains)
- Mais comme je suis pas bien sur de moin tentative GPT à vérifier

In [24]:
import re

# Regex du champ lexical République (simplifié ici)
# pattern_lexical = re.compile(r"\b(anti-?république(s)?|république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b", re.I)
# sinon :
pattern_lexical = re.compile(
    r"(?<!\w)(anti-?|pré-?|post-?)?(république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b",
    re.I,
)


# Regex des expressions à exclure
pattern_excl = re.compile(
    r"\bLes Républicains\b|Président de la République"
)  # ici garder la casse


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte"].apply(contains_lexical_outside_excl)

In [25]:
df[df["repu_match_valide"]].shape

(4592, 60)

In [26]:
# Version bourrin
# Champ lexical : True si le texte contient un terme du champ lexical
lexical_mask = df["Texte"].str.contains(pattern_lexical)

# Exclusion : True si le texte contient une expression à exclure
excl_mask = df["Texte"].str.contains(pattern_excl)

# On garde ceux qui matchent lexicalement mais ne contiennent pas d'exclusion
df["repu_match_simple"] = lexical_mask & ~excl_mask


/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_6079/55792929.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lexical_mask = df["Texte"].str.contains(pattern_lexical)


In [27]:
df[df["repu_match_simple"]].shape

(4114, 61)

In [28]:
import csv

df_match = df[df["repu_match_valide"]]
df_match.to_csv(
    "../data/df_repu.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)


In [29]:
df_match.shape

(4592, 61)

In [30]:
df_test = pd.read_csv("../data/df_match.csv")

In [31]:
df_test.shape

(4592, 61)